##Bring in libraries and dataframes and set indexes

In [1]:
import datetime
import pandas as pd
import numpy as np

In [2]:
#Read in data
AllData=pd.read_csv('K:\Rainshelter\StonySoilLysimeters\DownloadedData\StonyLysimetersCS650.dat', #specify file path for data to read in
                         parse_dates=True, #tell the function to parse date columns to datetime formats
                         dayfirst=True, #tell the function that the day is before the year in the data i.e format='%d/%m/%Y %H:%M'
                         skiprows = [0,2,3], #leave out rows 1, 3 and 4 which have redundant information
                         index_col = 0, #Use the first column, which is Date, as an index
                         na_values = 'NAN')

#Read in calibration coefficients calculated in C:\GitHubRepos\StonySoilLysimeters\Calibration\CS650Calibration.ipynb
SensorCoefficients = pd.read_pickle('C:\GitHubRepos\StonySoilLysimeters\Calibration\CS650Calibration.pkl')

#Bring in all index data
AllDataIndex=pd.read_csv('C:\GitHubRepos\StonySoilLysimeters\IndexFiles\StonyLysimetersWater&TempIndex.csv',
                         index_col = 0)

#Bring in index for horizion types
HorizonIndex=pd.read_csv('C:\GitHubRepos\StonySoilLysimeters\IndexFiles\HorizonIndex.csv', #specify file path for data to read in
                         index_col = 0, #Use the first column, which is Date, as an index
                         na_values = 'NAN')

#Apply indexes to data
AllDataTransposed = AllData.transpose()
AllDataIndexed = pd.concat([AllDataIndex,AllDataTransposed], axis=1)
AllDataIndexed.index.name='ColumnHeader'
AllDataIndexed.set_index(['Measurement','Depth','Gravels','Stones','Column','Sensor', 'MUX', 'Port','Units','Summary','Block','Treatment'], 
                        append=False, inplace=True)
AllDataIndexed.sort(inplace=True)
Data=AllDataIndexed.transpose()
Data.index = Data.index.to_datetime()  ## for some reason the concat function changes the data type on the date indes so need to change it back

In [3]:
max(Data.index)

Timestamp('2016-02-12 10:30:00')

In [4]:
Data.index.size

35036

In [5]:
#Read in data
TestRead=pd.read_csv('K:\Rainshelter\StonySoilLysimeters\DownloadedData\StonyLysimetersCS650.dat', #specify file path for data to read in
                         parse_dates=True, #tell the function to parse date columns to datetime formats
                         dayfirst=True, #tell the function that the day is before the year in the data i.e format='%d/%m/%Y %H:%M'
                         skiprows = [0,2,3], #leave out rows 1, 3 and 4 which have redundant information
                         index_col = 0, #Use the first column, which is Date, as an index
                         na_values = 'NAN')

In [6]:
TestRead

,RECORD,BattV_Avg,PTemp_C_Avg,"CS650(1,1)","CS650(1,2)","CS650(1,3)","CS650(1,4)","CS650(1,5)","CS650(1,6)","CS650(2,1)",...,"CS650(95,3)","CS650(95,4)","CS650(95,5)","CS650(95,6)","CS650(96,1)","CS650(96,2)","CS650(96,3)","CS650(96,4)","CS650(96,5)","CS650(96,6)"
TIMESTAMP,,,,,,,,,,,,,,,,,,,,,
2015-02-07 12:15:00,0,13.78,25.76,0.2238,0.0682,19.2086,11.8967,2.8121,1.1997,0.1797,...,15.4317,14.1124,2.9930,1.3092,0.2312,0.0959,16.2676,12.3125,2.8743,1.2800
2015-02-07 12:30:00,1,13.79,26.19,0.2236,0.0692,19.8048,11.8854,2.8121,1.2025,0.1795,...,15.4620,14.1122,2.9930,1.3092,0.2313,0.0952,16.2539,12.3208,2.8743,1.2781
2015-02-07 12:45:00,2,13.79,27.09,0.2237,0.0699,20.3794,11.8935,2.8133,1.2046,0.1794,...,15.4867,14.1117,2.9930,1.3094,0.2313,0.0954,16.2209,12.3182,2.8743,1.2787
2015-02-07 13:00:00,3,13.79,28.07,0.2235,0.0708,20.9051,11.8832,2.8133,1.2071,0.1793,...,15.5281,14.1135,2.9930,1.3089,0.2313,0.0953,16.2264,12.3201,2.8743,1.2783
2015-02-07 13:15:00,4,13.79,28.99,0.2234,0.0714,21.5109,11.8754,2.8133,1.2091,0.1792,...,15.5611,14.1152,2.9930,1.3085,0.2313,0.0955,16.2209,12.3175,2.8743,1.2789
2015-02-07 13:30:00,5,13.79,29.95,0.2235,0.0723,22.1396,11.8814,2.8144,1.2116,0.1791,...,15.5721,14.1121,2.9930,1.3093,0.2313,0.0955,16.1990,12.3179,2.8743,1.2788
2015-02-07 13:45:00,6,13.79,30.84,0.2233,0.0734,22.7667,11.8688,2.8144,1.2147,0.1793,...,15.5996,14.1128,2.9930,1.3091,0.2310,0.0952,16.1962,12.3039,2.8732,1.2781
2015-02-07 14:00:00,7,13.79,31.62,0.2231,0.0741,23.3926,11.8596,2.8144,1.2170,0.1793,...,15.6381,14.1082,2.9930,1.3102,0.2311,0.0948,16.2127,12.3096,2.8732,1.2768
2015-02-07 14:15:00,8,13.79,32.22,0.2232,0.0752,23.9509,11.8638,2.8155,1.2199,0.1791,...,15.6877,14.1083,2.9930,1.3102,0.2310,0.0953,16.1990,12.3036,2.8732,1.2782


In [7]:
from datetime import timedelta #Import function needed for doing date deltas 
Test = Data[:12585].index - timedelta(hours=1) 
Test

DatetimeIndex(['2015-02-07 11:15:00', '2015-02-07 11:30:00',
               '2015-02-07 11:45:00', '2015-02-07 12:00:00',
               '2015-02-07 12:15:00', '2015-02-07 12:30:00',
               '2015-02-07 12:45:00', '2015-02-07 13:00:00',
               '2015-02-07 13:15:00', '2015-02-07 13:30:00', 
               ...
               '2015-06-23 11:15:00', '2015-06-23 11:30:00',
               '2015-06-23 11:45:00', '2015-06-23 12:00:00',
               '2015-06-23 12:15:00', '2015-06-23 12:30:00',
               '2015-06-23 12:45:00', '2015-06-23 13:00:00',
               '2015-06-23 13:15:00', '2015-06-23 13:30:00'],
              dtype='datetime64[ns]', length=12585, freq=None, tz=None)

## Creat new data frame and apply calibration coefficients for each horizon type

In [8]:
VWC_uncalibrated = Data.VolumetricWaterContent
VWC_uncalibrated.columns = VWC_uncalibrated.columns.swaplevel('Depth','Sensor') #Make sensor the top level so index selection works properly
VWC_calibrated = pd.DataFrame(index = VWC_uncalibrated.index, columns = VWC_uncalibrated.columns)  #Set up empty dataframe with the same index and column structure as TDR data

for X in range (1,97): 
    SensorRef = 'S_' + np.str(X)
    Horizon = HorizonIndex.ix[X,'Horizon']
    Slope = SensorCoefficients.ix[Horizon,'Slope']
    Intercept = SensorCoefficients.ix[Horizon,'Intercept']
    VWC_calibrated[SensorRef] = VWC_uncalibrated.ix[:,SensorRef] * Slope + Intercept 

##Calculat plant available water for each horizon

In [9]:
PAW = pd.DataFrame(index = VWC_uncalibrated.index, columns = VWC_uncalibrated.columns)  #Set up empty dataframe with the same index and column structure as TDR data

for X in range (1,97): 
    SensorRef = 'S_' + np.str(X)
    Horizon = HorizonIndex.ix[X,'Horizon']
    LL = SensorCoefficients.ix[Horizon,'LL']
    PAW[SensorRef] = VWC_calibrated.ix[:,SensorRef] - LL 

PAW.to_pickle('.\DataFrames\LysoPAW_df')

## Export volumetric water content dataframes

In [10]:
VWC = VWC_calibrated
VWC.columns = VWC.columns.swaplevel('Sensor','Depth') # Get index back in the same order as the others
VWC.to_pickle('.\DataFrames\LysoVWC_df')

## Export Temperature data frame

In [11]:
Temp = Data.Temperature
Temp.to_pickle('.\DataFrames\LysoTemp_df')

## Export Electrical Conductivity Data frame

In [12]:
EC = Data.ElectricalConductivity
EC.to_pickle('.\DataFrames\LysoEC_df')

##Export Permittivity Data frame

In [13]:
Perm = Data.Permittivity
Perm.to_pickle('.\DataFrames\LysoPerm_df')

##Export Period Data Frame

In [14]:
Period = Data.Period
Period.to_pickle('.\DataFrames\LysoPeriod_df')

##Export Voltage ratio data

In [15]:
VR = Data.VoltageRatio
VR.to_pickle('.\DataFrames\LysoVR_df')

##Export Drainage Data

In [16]:
#Read in data
RawDrainageData=pd.read_csv('K:\Rainshelter\StonySoilLysimeters\DownloadedData\StonyLysimetersDrain.dat', #specify file path for data to read in
                         parse_dates=True, #tell the function to parse date columns to datetime formats
                         dayfirst=True, #tell the function that the day is before the year in the data i.e format='%d/%m/%Y %H:%M'
                         skiprows = [0,2,3], #leave out rows 1, 3 and 4 which have redundant information
                         index_col = 0, #Use the first column, which is Date, as an index
                         na_values = 'NAN')

#Bring in all index data
DrainageDataIndex=pd.read_csv('C:\GitHubRepos\StonySoilLysimeters\IndexFiles\DrainageIndex.csv',
                         index_col = 0)

#Apply indexes to data
DrainageDataTransposed = RawDrainageData.transpose()
DrainageDataIndexed = pd.concat([DrainageDataIndex,DrainageDataTransposed], axis=1)
DrainageDataIndexed.index.name='ColumnHeader',
DrainageDataIndexed.set_index(['Measurement','Gravels','Stones','Column','Sensor', 'MUX', 'Port','Units','Summary','Block','Treatment'], 
                        append=False, inplace=True)
DrainageDataIndexed.sort(inplace=True)
DrainageData=DrainageDataIndexed.transpose()
DrainageData.index = DrainageData.index.to_datetime()  ## for some reason the concat function changes the data type on the date indes so need to change it back

In [17]:
DR = DrainageData['Drainage']
DR.to_pickle('.\DataFrames\LysoDrainage_df')